This notebook runs the given model over all the benchmark sentences and show their alignment, stop-token prediction and spectrogram plot in order.

In [ ]:
cd ~/projects/

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import io
import torch 
import time
import numpy as np
from collections import OrderedDict
from matplotlib import pylab as plt

%pylab inline
rcParams["figure.figsize"] = (16,5)
# sys.path.append('/home/erogol/projects/') # set this if TTS is not installed globally

import librosa
import librosa.display

from TTS.models.tacotron import Tacotron 
from TTS.layers import *
from TTS.utils.data import *
from TTS.utils.audio import AudioProcessor
from TTS.utils.generic_utils import load_config
from TTS.utils.text import text_to_sequence
from TTS.utils.synthesis import synthesis
from TTS.utils.visual import visualize

import IPython
from IPython.display import Audio


In [ ]:
def tts(model, text, CONFIG, use_cuda, ap, figures=True):
    t_1 = time.time()
    waveform, alignment, spectrogram, mel_spectrogram, stop_tokens = synthesis(model, text, CONFIG, use_cuda, ap) 
    print(" >  Run-time: {}".format(time.time() - t_1))
    if figures:                                                                                                         
        visualize(alignment, spectrogram, stop_tokens, text, ap.hop_length, CONFIG, mel_spectrogram)                                                                       
    IPython.display.display(Audio(waveform, rate=CONFIG.audio['sample_rate']))  
    out_path = '/home/erogol/Dropbox/AudioSamples/benchmark_samples/'
    os.makedirs(out_path, exist_ok=True)
    file_name = text.replace(" ", "_").replace(".","") + ".wav"
    out_path = os.path.join(out_path, file_name)
    ap.save_wav(waveform, out_path)
    return alignment, spectrogram, stop_tokens

In [ ]:
# Set constants
ROOT_PATH = '/home/erogol/test_model/'
MODEL_PATH = ROOT_PATH + 'best_model.pth.tar'
CONFIG_PATH = ROOT_PATH + '/config.json'
OUT_FOLDER = ROOT_PATH + '/test'
CONFIG = load_config(CONFIG_PATH)
use_cuda = True

In [ ]:
from utils.text.symbols import symbols, phonemes

# load the model
num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)
model = Tacotron(num_chars, CONFIG.embedding_size, CONFIG.audio['num_freq'], CONFIG.audio['num_mels'], CONFIG.r, attn_windowing=False)

# load the audio processor
# CONFIG.audio["power"] = 1.3
CONFIG.audio["preemphasis"] = 0.97
ap = AudioProcessor(**CONFIG.audio)         


# load model state
if use_cuda:
    cp = torch.load(MODEL_PATH)
else:
    cp = torch.load(MODEL_PATH, map_location=lambda storage, loc: storage)

# load the model
model.load_state_dict(cp['model'])
if use_cuda:
    model.cuda()
model.eval()
print(cp['step'])



### Comparision with https://mycroft.ai/blog/available-voices/

In [ ]:
model.eval()
sentence =  "It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent."
model.decoder.max_decoder_steps = 1000
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap, figures=True)

In [ ]:
sentence = "Be a voice,not an echo."  # 'echo' is not in training set. 
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "The human voice is the most perfect instrument of all."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "I'm sorry Dave. I'm afraid I can't do that ."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "This cake is great. It's so delicious and moist ."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "This cake is great. It's so delicious and moist."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

### Comparison with https://keithito.github.io/audio-samples/

In [ ]:
sentence = " Generative adversarial network or variational auto-encoder."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "Scientists at the CERN laboratory say they have discovered a new particle."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "Here’s a way to measure the acute emotional intelligence that has never gone out of style."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "President Trump met with other leaders at the Group of 20 conference."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "The buses aren't the problem, they actually provide a solution."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

### Comparison with https://google.github.io/tacotron/publications/tacotron/index.html

In [ ]:
sentence = "Generative adversarial network or variational auto-encoder."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "Basilar membrane and otolaryngology are not auto-correlations."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = " He has read the whole thing."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "He reads books."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "Thisss isrealy awhsome."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "This is your internet browser, Firefox."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "This is your internet browser Firefox."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "The quick brown fox jumps over the lazy dog."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "Does the quick brown fox jump over the lazy dog?"
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "Eren, how are you?"
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

### Hard Sentences

In [ ]:
sentence = "Encouraged, he started with a minute a day."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "His meditation consisted of “body scanning” which involved focusing his mind and energy on each section of the body from head to toe ."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "Recent research at Harvard has shown meditating for as little as 8 weeks can actually increase the grey matter in the parts of the brain responsible for emotional regulation and learning . "
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "If he decided to watch TV he really watched it."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
sentence = "Often we try to bring about change through sheer effort and we put all of our energy into a new initiative ."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
# for twb dataset
sentence = "In our preparation for Easter, God in his providence offers us each year the season of Lent as a “sacramental sign of our conversion."
align, spec, stop_tokens = tts(model, sentence, CONFIG, use_cuda, ap)

In [ ]:
!zip benchmark_samples/samples.zip benchmark_samples/*